In [1]:
import pandas as pd
import sqlite3
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from classification_1_parsing_parrotorno import make_dictionnary
#Style of the figure
plt.style.use('seaborn-whitegrid')

In [2]:


# Create a SQL connection to our SQLite database
con = sqlite3.connect("DATABASES/project.db")

cur = con.cursor()

ana_df = pd.read_sql_query("SELECT * from classification_1_analysis", con)
map_df= pd.read_sql_query("SELECT * from mapping_cites", con)

In [3]:
fig, ax = plt.subplot(7, 8,figsize= (8,6), sharey = True, sharex = True)

bag_of_words=make_dictionnary()
for i in range(1,len(map_df)+1,1):
    sbplt = ax[i%8, math.floor(i/8)]
    words=bag_words(i)
    list_tot={}
    for word in words :
        total=ana_df[word].sum()
        list_tot.append(total)
    
    sbplt.bar(words, List_tot)
#     df = pd.DataFrame({'lab':['A', 'B', 'C'], 'val':[10, 30, 20]})
#     sbplt = plt.plot.bar(x='lab', y='val', rot=0)
    title=map_df.at(i, "scientific_name_cites")
    sbplt.set_title(f"{title}")
    plt.xlabel('Date')
    plt.ylabel('Number of pageviews')
fig.tight_layout(pad=2)

SyntaxError: invalid syntax (<ipython-input-3-bfe56f476c0c>, line 14)